In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)
/usr/local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [5]:
!pip install bcolz


    100% |████████████████████████████████| 1.3MB 1.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/e9/84/eb/f8f3caa627bb01ebc96034c3411f59870951246e5873b3f4c7
Successfully built bcolz


In [6]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)
/usr/local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [1]:
%cd /root

/root


In [2]:
%mkdir ~/.keras

In [12]:
% cd ~/.keras

/root/.keras


In [3]:
!echo '{"image_dim_ordering": "th", "epsilon": 1e-07, "floatx": "float32", "backend": "theano" }' > ~/.keras/keras.json

In [4]:
% cd /output




/output


In [7]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [8]:
#path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open("duda.txt").read().lower()
print('corpus length:', len(text))

corpus length: 139932


In [9]:
!tail {"duda.txt"} -n25

In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 78


In [11]:
chars.insert(0, "\0")

In [12]:
''.join(chars[1:-6])

'\n\r !"%()+,-.0123456789:?abcdefghijklmnopqrstuvwxyz\x80\x81\x82\x84\x85\x86\x87\x92\x93\x99\x9a\x9b\x9d\x9e\xa1\xad\xb3\xba\xbb\xbc\xbf'

In [13]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [14]:
idx = [char_indices[c] for c in text]

In [15]:
idx[:10]

[77, 69, 71, 43, 50, 25, 38, 39, 47, 38]

In [16]:
''.join(indices_char[i] for i in idx[:70])

'\xef\xbb\xbfszanowni pa\xc5\x84stwo marsza\xc5\x82kowie i marsza\xc5\x82kowie seniorzy,\r\nszanowna'

## Preprocess and create model

In [17]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 139893


In [18]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [19]:
sentences.shape, next_chars.shape

((139891, 40), (139891, 40))

In [20]:
n_fac = 24

In [21]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        BatchNormalization(),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [22]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam() )

## Train

In [23]:
def print_example():
    seed_string="szanowny panie marszałku i szanowni zebr"
    for i in range(800):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [24]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=32, nb_epoch=1, verbose = 2)

Epoch 1/1
431s - loss: 1.3396


In [25]:
print_example()

szanowny panie marszałku i szanowni zebronnych.

ci, którzy od tym pamiętać, bo to będą chciałbym życzyć ‒ solidarność” tego, jakie są dzisiaj mogę wolny”. tylko także z mojej drodzy się wyciągowaliśmy – jako ministrowie,
szanowni państwo!

panowie gprofesorowi.

to, że efekt synergii nato mogłyby z krajską.

jest potrzebne zaproszenie tego, o czym odrazić, czy powinna być państwa i pomysły miejscu poznaliśmy do zawodu, które mówię „budować konsjałbym życia, jest tam, elemu przede wszystkim powinniśmy tego wielkim do tego,
ci, którzy w opiście możliwość w naszych strategicznymi z naszych odpowiadali do szerokim środowisku just do apilku w życiu chodzieni są przed tym wojne reklaru każdego perspektywie nasz dziedzina w budowania, które musimy mówiąc o groś dobre prz


In [26]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=32, nb_epoch=1, verbose =2)

Epoch 1/1
431s - loss: 0.9989


In [27]:
print_example()

szanowny panie marszałku i szanowni zebrony i chciałem, przez mamy widzieli, że to polska z całą pewnością jest 112, jeżeli się mogą naszego kraju. to właśnie przy tymi, którzy w których będą przestrzeganie. tak jak państwo z premier, obecność w ziemię przyczyniając państwu o majątku chrztu, które planuje są poziom życia każdego państwa, dla mnie ten ówczesnego szczególnie i pana prezydenta profesor lech kaczyński w naszym kraju, w starożytnym sąsiadów i samorządowców, w których nie było, jako wielkiego wyłonić do wątkowy.

Życzę gań, bo „solidarność w polsce w polsce. może nie tylko do tego, aby w sferze publicznego. ale to, proszę państwa, doskonale także edmokatury, przeszła i samorządu – tego, co produkt jest także poprzez pracę sprosy, pan węgry "mówić o tych kra


In [28]:
model.optimizer.lr=0.001

In [29]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
256s - loss: 0.8895


In [30]:
print_example()

szanowny panie marszałku i szanowni zebrali niezwykle ważną rolę ważny dla prezydenta lecha kaczyńskiego, które myślę, dziękuję panu promowaniu tradycją. dziękuję za współpracy z ukrainą i zachodem.
uroku w jakimś sensie treścią: „nie wyjdziemy z tego, że swoimi funkcjonującym, zarówno zainteresowali się, że w ciągu ostatniego pozwolu 1956 roku, gdy one są różne, które sprzemieszczania nasze przestrzegane, i jest to, dlaczego oni nasze pięknym obywateli doświadczeniem prawniczy to dzisiaj naszym polskimi firmami” ‒ było też ko trudnym państwu życzyć państwu powiedzieć, że są już podkreślając ją konkretne, jako prezydent rzeczypospolitej dalszy doscy dla polskiej gospodarki, są w sposób najważniejszym elementem pan profesor absolutnie żadną miasta i rodzin, ich przepodatkowa


In [31]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
256s - loss: 0.8447


In [32]:
print_example()

szanowny panie marszałku i szanowni zebrani w ramach firm, sposób wzajemnie kraje na tym osobista nas największych się zaszyfrowane, i jest właśnie ten efekt synergii ponieważ tak jak wspomnieć szczegóły nie miałem stanie się konkurencyjność w procesach politycznych informacji systemowej czy ‒ właśnie kwestia był wtedy powinno być przez państwa środowisko, które swoimi unii europejskiej. wolność rozumianą wspólnotę europy, abyśmy mówili o tym, czego dziś mówiło, że ta polska b. też wielu tradycja demokratycznych adwokatów, które być wolność. wcześniej, o działalność rozwoju budzi w polsce, w imię zasady – tego, co stanowi jednoczonych, przed laty mówią – przyszła nastąpiła europejskiej. wolność rozumieli, że ta „myśląca nigdy nie byliśmy razem na to sojuszniczych ukrai


In [33]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
257s - loss: 0.8128


In [34]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
257s - loss: 0.7871


In [35]:
print_example()

szanowny panie marszałku i szanowni zebronić w pracownika międzynarodowego unikania opodatkowania to z nas nie wygodnie z władze adwokatury, które się na każdym państwie dzisiaj, następnym obiekcie, będziemy mówili o tym, czy ze styku przewodniczący nie będzie wysuniętej obecności służyli czy także możemy mówić „władz regionalnych, siłami, w których także odstrzegane, aby był kilkakrotnie wyższe.

słusznie padtnierze,
wszyscy dla tych, którzy są obecni tutaj z nami.

ale to nie tylko  samorząda regionalnych, w zakresie strzeżenia – dziękuję panu prezydentowi piotrkowa trybunalskiego, obrońcy gospodarczego rzeczypospolitej także jaką kolejność rozumiać w naszym kraju, w których dochód per capita jest jej prezydent rzeczypospolitej.
szanowni państwo wiedzą, tę niezwykle istotny


In [38]:
print_example()

szanowny panie marszałku i szanowni zebrełiom i podoficerowie,
szanowni państwo ministrowie,
szanowni państwo ministrowie,
szanowni państwo, nominowane, za co gorąco dziękuję.
podkreślają, był to w związku z tym też warto odjakie stanowisko, ale chcę o tym pamiętać zwłaszcza młodego państwa polskiego biznesu wemu, aby do budowaniu państwa z państwa jedną prośbą: może niektórzy z nich było też takie hasło „dobre pomysły, które być może jest niezwykle ważną rolę w państwie wzmacnianiu gospodarczym rzeczypospolitej, będziemy mówili o tym, czego tak i kancelarii prezydenta lecha kaczyńskiego, jako prezydent rzeczypospolitej, że te wielkie zadanie zasad i przepisów prowadziły do wzmocnienia dobrych warunków, żeby jest moje osobiste doświadczenia politycznych – bo przecież ostatnio


In [36]:
model.optimizer.lr=0.0001

In [37]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
257s - loss: 0.7662


In [39]:
print_example()

szanowny panie marszałku i szanowni zebronych w ręku. właśnie dokładnie tu, gdzie w 1970 roku, gdy te niezwykle ważna i niepodległość, w walce o wolność.

najcenniejszą rzeczą do budowanie libla sprawy. prawne. i już się ucało, bo to zwyczajne. dziękuję panu prezydentowi również przyniesie polsce jest ogromnie się cieszę, że tych młodych przedsiębiorcy, abyśmy do tego przemysłu kraju, na którego wiedzą, że pan profesor Ćwiąkalski był 24 grudnia 1918 roku, 97 lat od naszego kraju.

chcę z tego, że spotkał mnie dzisiaj to, ocalości opinii na tej sali, i robi także ich oceny i znaczenia konfliktu jest w zasadzie u samego początku, bo jest długa. bardzo serdecznie chciałbym, żeby za kilka lat choćby być dwoistę do absolutnie radek był pan prezydent z małżonką dzisiaj z nami,
ci, k


In [40]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=128, nb_epoch=5, verbose = 2)

Epoch 1/5
185s - loss: 0.7283
Epoch 2/5
185s - loss: 0.7104
Epoch 3/5
185s - loss: 0.6994
Epoch 4/5
185s - loss: 0.6891
Epoch 5/5
185s - loss: 0.6798


In [41]:
print_example()

szanowny panie marszałku i szanowni zebrali po imieniu gospodarczym rzeczypospolitej i dla mnie wielkie wrażenie, bo polska tradycja, dobrze funkcjonująca firmach, dla elementów naukowych, które tutaj na scenie – wieloletni.

dzięki istnienie także tutaj z nami i delegacji węgierskiej przez pracownika, przeciętnego obywatela będzie działał etycznie, starało się nam odzyskać, choć na szczęście zachowała się w rzeczypospolitej zaproszę naszych spotkań „solidarność”, całego związku zawodowego, eneraz ukraińskim bezpieczeństwa i higieny pracy są niezagrożone. oczywiście zapraszam do rzeszowa, bardzo serdecznie dziękuję pani premier beaty szydło i większości parlamentarnej. to, proszę państwa, do którego w jej od samego początku, rozwijając ich przedsiębiorczości, dla przeciętnego ob


In [43]:
model.optimizer.lr=0.00001

In [44]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=128, nb_epoch=2, verbose = 2)

Epoch 1/2
185s - loss: 0.6700
Epoch 2/2
185s - loss: 0.6616


In [45]:
print_example()

szanowny panie marszałku i szanowni zebryli, w końcu oddał życie, do tego, by się spotykając, i wypracowaliśmy zgodnie i spokojnie krzepnąć. wciąż obecne w jej systemie ekonomiczną oraz doświadczenie łączyła z otwartością niepodległość, w walce o budowanie potencjału tutaj, w przypadku większości parlamentarnej, aby stwarzać dla nas, polaków, było to doskonale wiedzą w tym rozwoju z jednej strony wszystko, co zdarzy się w tej czołówce była. jedną z naszych bohaterów narodowych, ale ‒ czy program dziękuję za chwilę to jest niezwykle ważne. dziękuję.
szczęść boże,
czcigodny i państwa zakłady pracy, jednym z naszych start-upów wielkich podmiotów gospodarczych dzisiaj rozmawiali właśnie oddania i tej przyszłości oraz aby nie tylko wyrazem solidarności, która jest egzekwowana, 


In [46]:
print_example()

szanowny panie marszałku i szanowni zebrości obrony terytorium sojuszu opartej wyłącznie o władze samorządowe, sobie nasze narody i państwa środowisku adwokaci, którzy w czasach terroru i zniewolenia dali wspaniałe dowody męstwa się nie było. chcę to bardzo mocno podkreślanie tego, co można powiedzieć, że – rodziła się w specyficznym sektorze, gdzie mogą swobodnie działać, gdzie praca będzie wystawiało zarazem opinię także władze publiczna by nie było to, aby ci, którzy za kwestie dyscyplinarną w ramach tego przemysłu gospodarczego, co w polsce, bo stawić czoła przed bruktowych ludzi?!

-często mówi mojego, że to w ogóle nie było dla wszystkich leuach. to wszystko, co było wtedy w jakimś sensie treścią ich życia: dla konferencja oraz narodowych. suwerenne państwo.

przystąpienie


In [47]:
print_example()

szanowny panie marszałku i szanowni zebrazy w życiu każdego obywatela. to jest, proszę państwa, to prezesi,
wszyscy dostojni zgromadzeni goście,
czysto politycznych, czyli organizację tego spotkania: alcide de gasperiego i roberta schumana prowadzi nas – także i w oparciu o oceny wystawiane przeze mnie niezwykle konkurencyjność w znaczeniu przez króla w walce o wolność rodaków. gratuluję białostocczanom, którzy walczyli o silne państwo. pracowaliśmy rocznicę rządu prezydenta w zeszłym roku obchodzimy 1050-lecie chciałbym tchnąć nowego ducha, górnicy są ludźmi wielkiego formatu. rodzinie oni wśród polskich obywateli uratowali także i w mężczyznach, w których bardzo często bardzo serdecznie dziękuję za przybycie do nas, do polski – wzajemne gwarancje bezpieczeństwa. gratuluję bieżeli co


In [42]:
model.save_weights('char_dudaGPUTheano.h5')

In [48]:
model.save_weights('char_dudaGPUTheano2.h5')

In [49]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=256, nb_epoch=2, verbose = 2)

Epoch 1/2
142s - loss: 0.6405
Epoch 2/2
142s - loss: 0.6296


In [50]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=128, nb_epoch=2, verbose = 2)

Epoch 1/2
185s - loss: 0.6427
Epoch 2/2
185s - loss: 0.6387


In [51]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=256, nb_epoch=2, verbose = 2)

Epoch 1/2
142s - loss: 0.6171
Epoch 2/2
142s - loss: 0.6080


In [52]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=512, nb_epoch=2, verbose = 2)

Epoch 1/2
135s - loss: 0.5962
Epoch 2/2
135s - loss: 0.5901


In [53]:
model.save_weights('char_dudaGPUTheano3.h5')

In [54]:
def print_example():
    seed_string="szanowny panie marszałku i szanowni zebr"
    for i in range(2400):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [55]:
print_example()

szanowny panie marszałku i szanowni zebrazy są pewne obiektywne kryteria, które budują zaufanie społeczeństwem, zanszela w skali świata krytyki, jak też sympatię szerokiej publiczności. rząd rzeczypospolitej, o dobrej zmianie, to mówimy, że te zmiany właśnie jego. nawet ci najbliżsi, którzy przyprowadzili tutaj swoje dzieci, tak pięknie machające naszymi polskimi barwami. bardzo serdecznie zapraszam do rzeszowa, bardzo serdecznie zapraszam do rzeszowa, bardzo serdecznie panom profesorom gratuluję państwu jeszcze raz zachęcam do tego, aby zdać później egzamin i móc się poziom życia każdego polaka. głęboko jestem przekonany, że dzięki współpracy państw europy centralnej, wysuniętej obecności (efp), ale też bardzo ważne jest, by dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkcjonująca firma, dobrze funkc

In [56]:
print_example()

szanowny panie marszałku i szanowni zebrali na najbliższe lata powinno być zacieśnienie naszej efektywny aparat państwo. nie skazuje ich na drakońskie kary, bo one ‒ tak jak zresztą też wszystkim tym, którzy w tamtych dniach potrafili trwać z podniesioną głową.

ogromnie się cieszę i ogromną satysfakcją jest to, że ona jest długa. najważniejszym elementem polityki realizowanych jest w tej chwili otwarty, zawsze! jeżeli zaś razem, że można było wcześniej w ii rzeczypospolitej zwrócić się o objęcie tej nagrody, z roku w poznaniu, a następnie rewolucja węgierskiej pomocy zdołaliśmy się w państwie jest niezbędny.

najcenniejszą rzeczą dla narodu jest wolne, suwerenne i niepodległość, odzyskaliśmy wtedy tej pamięci. ona została przez wszystkich, którzy być realizowane - obrońcach procesowych za czasów. związane jest ono z działalnością tych podmiotów, które w polsce mogłyby być adwokatem. to są kryteria jasne, klarowne, może nie zadrżały budynki i okna, ale z drugiej strony także polskie pr

In [57]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=512, nb_epoch=10, verbose = 2)

Epoch 1/10
134s - loss: 0.5858
Epoch 2/10
135s - loss: 0.5830
Epoch 3/10
135s - loss: 0.5800
Epoch 4/10
135s - loss: 0.5774
Epoch 5/10
135s - loss: 0.5749
Epoch 6/10
135s - loss: 0.5724
Epoch 7/10
135s - loss: 0.5706
Epoch 8/10
135s - loss: 0.5683
Epoch 9/10
136s - loss: 0.5669
Epoch 10/10
136s - loss: 0.5646


In [58]:
model.optimizer.lr=0.000001

In [59]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=512, nb_epoch=1, verbose = 2)

Epoch 1/1
135s - loss: 0.5623


In [60]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=512, nb_epoch=4, verbose = 2)

Epoch 1/4
135s - loss: 0.5594
Epoch 2/4
135s - loss: 0.5582
Epoch 3/4
134s - loss: 0.5561
Epoch 4/4
136s - loss: 0.5547


In [61]:
model.save_weights('char_dudaGPUTheano4.h5')

In [62]:
print_example()

szanowny panie marszałku i szanowni zebrali w maju br. skupi się na tych administracji, prawda, po to, żeby uwolnić energię tych najmłodszych, tych, którzy chcieli odzyskać wolność. ten krzyż jest w gdańsku, w gdyni, gdzie w 1970 roku ludzie walczyli ‒ godność, solidarność, sprawiedliwość. ten krzyż jest w gdańsku, w gdyni, gdzie w 1956 roku adwokaci, którzy zginęli, walcząc o niepodległość. właśnie dlatego, że umacniając tym samym jego niepodległe państwo polskie – zginęli w obronie ojczyzny w 1920 roku. nie uwzględnia głębokich zmian w sfariżonych, z kanady, z innych krajów, ich wzajemne spotkanie z panami na moment zaprosiła. dziękuję za przybycie, jeszcze raz bardzo serdecznie dziękuję społeczną i patriotyczną, która jest egzekwowana. ale ja, proszę państwa, chcemy budować naszą własną innowacyjną gospodarkę. to zryw poznaniaków w ’56 roku – ale byli sami. to zryw studentów w ’68 roku – ale obok wrażliwości w kobietach także jest i ogromna. to świadczy o silnych więzach, to świadcz

In [63]:
print_example()

szanowny panie marszałku i szanowni zebrali w ramach życiowych.

jako prezydent rzeczypospolitej, ten zeszły rok był rokiem oczywiście zapraszając jednocześnie w świadomości społeczności naukowej.

jestem przekonany, że możemy razem stworzyć ramy dla państw wschodniej flanki. szczyt zmienił paradygmat bezpieczeństwa. z tej pełnionym procederem to olbrzymie wyzwanie, wymagające solidarności. to dzisiaj jest przecież ten szczególny dzień, który jest szczególna i niezwykła – ale obok wrażliwości w kobietach tę siłę wzajemnie, ale ona budzi siłę w całym otoczeniu, także i w mężczyznach. mogę  o tym, że przewodnicząca niszczy tkankę społeczną i patriotyczną, która również jest elementem tej ogólnej sytuacji – najlepszy.

trzy takie wyjazdy miały miejsce: to były szwajcarii, szczególnie się w tej chwili duchem, bo musiał się udać na spotkanie z panami na moment zaprosiła. dziękuję za przybycie, dziękuję panu prezydentowi i jego małżonce za przybycie przedstawiciele władz państwowych, to że s

In [64]:
print_example()

szanowny panie marszałku i szanowni zebrany przez to, że mogliśmy usiąść, porozmawiać w dobrej atmosferze, rzeczowej, merytorycznej, o ważnych dla siebie chwilach podejmuje refleksję nad własną tożsamością – podobnie też nasza wspólnotą. już nie byli podzieleni na studentów, robotników, bo jeszcze nie rozumieliśmy usiąść, porozmawiać w dobrej atmosferze, rzeczowej, merytorycznej, o ważnych dla wielkopolska – węgrzy zniszczyli polską młodzież potęgę i ogromną prosperity państw europy zachodniej. to jest to, dlaczego pkb na głowę, czyli tzw. per capita, jest tam dzisiaj tak ogromne. i to jest właśnie nasza historia. historia, o której przecież uczestniczyli w akcji ratowania obywateli uratowali też o lepsze miejsce na tę debatę niż szkoła główna handlowa – najstarsza i najbardziej intensywnych, jakie obecnie realizujemy. podpisaliśmy kilkaset umów i przyczyniając się bez wątpienia, jeszcze raz chylę czoła przed grobami poległych za wolność. nie byli jednak w stanie powstrzymać zwykłych p

In [65]:
print_example()

szanowny panie marszałku i szanowni zebrany w niemieckim obozie koncentracyjnych. to wszyscy polscy adwokaci, którzy spotkaliśmy się tutaj o początku lat 90. mówił – będzie pozytywny, będzie to potrzeba czy to wyjścia z inicjatywą legislacyjną, a więc ustawodawczą, to ja jako prezydent rzeczypospolitej do dzisiaj i to jest wielkie zadanie, które uda nam się wszystkim komunikacyjnych, które są tworzone.

może nie zaprzepaścić silnej więzi transatlantyckiej. jej treścią powinna być rzetelnie egzekwowana. ale ja uważam, że jesteśmy wiarygodnym i godnym zaufania członkiem sojuszu. bezdyskusyjne są pozytywne zmiany w siłach zbrojnych.

i tutaj dwie postaci, które należałoby wspomnieć ‒ wiele kopalni rozbiły czołgi, a potem braku prawdziwej niepodległego państwa. a mianowicie to, że ona jest długa, bo jest długa. najważniejsze nie jest to, kim, byli poddani, niszczyli wolność węgierską ‒ nie udało się udowodnić tę innowacyjną gospodarkę. to także zwykli rzemieślnicy, to wielu młodych ludzi z

In [66]:
print_example()

szanowny panie marszałku i szanowni zebrego i roberta schumana prowadzi nas – współczesnego rozdziału historii najnowszej polski.

wierzę, że ta praca buduje ducha patriotyczna i historyczna. to jest właśnie dokładnie taka płonąca lampka oświaty, który państwo wiedzą, tak jak musiały walczyć pokolenia naszych prapradziadków i pradziadków przez 123 lata, kiedy przeszliśmy do prawdziwej ofensywy? tym, którzy nie chcieli dłużej trwać w odrętwieniu, w pseudodobrobycie, którego tak silny nacisk na start-upy, dlatego obecność tutaj rapiej tych przyjaciół, którzy wtedy o polskę walczyli.

kiedy 25 lat temu podejmowaliśmy decyzję o uznaniu niepodległości jest zatem sprawny i efektywny aparat państwowy. nie istnieje żadną metodą na polskim majątku. to było jedno zjawisko, dochody budżetowe z tytułu podatku cit w polsce mówiło: „tak, mój pracodawca to pracodawca wzorcowy, godny wszystkie dobre pomysły, które wystarczy spełnić do tego – jeżeli samorządowi przed momentem pan prezes, byłem po raz p

In [67]:
print_example()

szanowny panie marszałku i szanowni zebrała w steżniejszym elementem inwestycyjnym jest właśnie nasza historia. historia, o której musimy pamiętać. ale nie wolno nam także i za liczne role filmowe, zdobyła zarówno uznanie świata krytyki, jak też sympatię szerokiej publiczności. i i powiem, proszę państwa, było tak cenne i co zawsze ‒ obok boga, honoru i ojczyzny ‒ wypisane było na naszym wspólnym wysiłkom udało się w 1918 roku. i wtedy słyszałem to wystąpieniu pan prezydent profesor lech kaczyński. ja uważam osobiście, że dostęp do zawodu adwokackiego, jako prezydent rzeczypospolitej ten aspekt dzisiejszej konferencji naukowej.

jestem przekonany, że ten kongres będzie się do tego przyczyniał. jeszcze raz wszystkim przestrzegany. warto przypomnieć słowa marszałka piłsudskiego, który za tę walkę i za tę pomoc, którą państwu prowadzona wspólna polityka ze strony ówczesnego rządu platformy obywatelskiego poprzez rozwój poczucia adekwatności wynagrodzenia do wykonywanej pracy utrzymywać ro

In [68]:
print_example()

szanowny panie marszałku i szanowni zebrany w niemieckim obozie koncentracyjnych i w innych miastach. to wszystko, co zdarzy się w przyszłości. otóż chciałbym tchnąć nowego ducha, górnicy są ludźmi – niezwykle ważne. dziękuję panu prezydentowi,
szanowni państwo ministrowie,
panie marszałku województwa podkarpackiego za to, że mogłem objąć ten kongres honorowym patronatem, zapewniam panie, że choćby był opór, to kobieca wrażliwość. kiedy wychodzili z więzienia, wracali z powrotem do działalności gospodarczego rozwoju, będącym zakładowego brutto, jeżeli chodzi o politykę działania w związku z tym także i jej prestiż. otóż, proszę państwa, w izraelu, gdzie mogli bronić w procesach politycznych, ale także – przyznajmy – realizacją polskiego interesu narodowego. rozwój każdego państwa. a mianowicie: ambicji tego, aby zawsze odnosiło ‒ i dla polskiego państwa. nasze wsparcie dla reformy administracyjnej i samorządowej na ukrainie zwyciężyła demokracja, nikt z nas nie przypuszczał, że niespeł

In [69]:
print_example()

szanowny panie marszałku i szanowni zebrego kanalizyka jest na przestrzeni tych ostatnich obrońców suwerennego i najpiękniejszego myślenia: solidności. pełni ona rolę najważniejszego międzyresortowego koordynatora narodowej – zwłaszcza ludzi, ale także – a może przede wszystkim, ale i polacy wiązali tak ogromne nadzieje na wolność, na sprawiedliwość, a to oznacza również snuć plany co do osobistego rozwoju, coraz doskonalszych pracowników i coraz lepszych wyników. gratuluję i dziękuję za tę dyskusję, bo – jak państwo słyszeli: „zapraszamy do inwestowania firmy z nimi walczyli, nie byłoby dzisiaj z nami pojechali. mam nadzieję – że będą to także w moim przekonaniu konieczne korekty, jakie powinny być, jeżeli chodzi o postanie przetrwali.

to, że zależy powinna być zarówno na węgrzech, jako polacy bez wahania. czyniliśmy to jako polacy bez wahania. czyniliśmy to bowiem dlatego, że pan prezydent jános Áder zadeklarował, że można traktatu waszyngtońskiego.

w ciągu ostatnich lat dzięki wkł

In [70]:
print_example()

szanowny panie marszałku i szanowni zebrady mówić o głębokiej, wielowiekowej czy ‒ jak niektórzy mówią – sięgnąć, którą moi rodacy wspinali sobie w ubrania na znak solidarności, która w istotnym stopniu uwłaszczyła się na świecie, dają sobie radę, a przynajmniej większość z nich daje sobie radę – i to bardzo często i podkreślało to słowo: nomenklatura, która w istotnym stopniu.

ekscelencjo, panie prezydencie!
szanowni państwo!

kiedy 25 lat temu podejmowaliśmy decyzję o uznaniu niepodległości ukrainy, uczyniliśmy to bowiem dlatego, że umieją. i dlatego, że umieją. i dlatego, że zależało nam, aby jednym z naszych strategicznych priorytetów. i muszę państwu powiedzieć, że jeszcze jeden element, który schwytany przez niemców, więziony i torturowany nigdy nie walczyć, tak jak musiały walczyć pokolenia naszych prapradziadków i przyczyniając się bez węgier, żona bolesława chrobrego. to potem już absolutnie udokumentowany. jan długosz w swoich kronikach pisze przy nich było codziennością.

o

In [71]:
print_example()

szanowny panie marszałku i szanowni zebrości europejskiej.

- dwa podstawowe elementy, jakie muszą być przygotowani  nie tylko wyrazem solidarności ze społeczeństwem, z narodem ukraińskim, z jego dążeniami. gdy w 2005 roku ukraińskie zadanie dzisiaj przede wszystkim ludziom, którzy w tym rozwoju uczestniczą swoją codzienną pracą. wspólną zamożność to chciałem, proszę państwa, ten kolorowy, odważny młody las, o którym mówi się, że należy do ściany wschodniej ‒ przecież wszyscy pamiętamy, że możemy się tutaj spotkać, że mogę ich uhonorować.

cieszę się, że nad problemem tak skomplikowanym będą dzisiaj wspólnym wysiłkom udało się walczyć o swój osobisty i rodzinny sukces, przyczyniając się bez wątpienia do budowania polskiego państwa ‒ skutek jak najbardziej pomyślnej, bezpiecznej przyszłości w sprawach, w których patrzymy także w tę wielką historię, tę najsilniejszą, do tego, aby realizować ekspansję zagraniczną.

proszę państwa, dwie wspomniane przeze mnie niezwykle ważne i co w dużym s